<img src="Images/GT2RBFnn.png">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import trange
from kmeans_pytorch import kmeans
device = 'cuda:0'#'cpu'#
from tensorboardX import SummaryWriter
import torch.nn.functional as F

# import matplotlib
# matplotlib.use("TkAgg")
import matplotlib.pyplot as plt



In [ ]:
class MyDataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return self.x.size(0)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return (x.type(torch.double), y.type(torch.double))
    

def error(a,b):
    return (0.5*torch.mean((a-b)**2))

In [29]:
from GT2RBFnn import GT2RBFnn

In [ ]:
# import numpy as np
# t = np.linspace(0, 2*np.pi, 100)
# x = np.array([np.sin(t)**2, np.cos(t)**3, np.sin(-t)**3]).T
# y = np.array([np.sin(-t), np.cos(t)**7]).T

# X_train = torch.tensor(x).to(torch.double).to(device)
# Y_train = torch.tensor(y).to(torch.double).to(device)

# X_test = X_train
# Y_test = Y_train

# model = GT2RBFnn(x.shape[1], 7, y.shape[1], 5).to(torch.double).to(device) #5,7,11
# model.Train(X_train, Y_train, Epochs = 90, batch_size=32, init_flag = True, lr = 0.01)
# y_hat = model.forward(X_train)

# plt.subplot(2,1,1)
# plt.plot(t, y[:,0])
# plt.plot(t, y_hat[:,0].cpu())
# plt.subplot(2,1,2)
# plt.plot(t, y[:,1])
# plt.plot(t, y_hat[:,1].cpu())
# # plt.show()

In [ ]:
import scipy.io as sc
import mat73
# '''
device2 = 'cpu'

temp1 = mat73.loadmat('X_all_30.mat')
temp2 = mat73.loadmat('Y_all_30.mat')

X_train = torch.tensor(temp1['X']['train'].T).to(torch.double).to(device2)
Y_train = torch.tensor(temp2['Y']['train'].T).to(torch.double).to(device2)
X_test = torch.tensor(temp1['X']['test'].T).to(torch.double).to(device2)
Y_test = torch.tensor(temp2['Y']['test'].T).to(torch.double).to(device2)

# temp1 = sc.loadmat('X.mat')
# temp2 = sc.loadmat('Y.mat')
# X_train = torch.tensor(temp1['X']['train'][0][0].T).to(torch.double).to(device)
# Y_train = torch.tensor(temp2['Y']['train'][0][0].T).to(torch.double).to(device)
# X_test = torch.tensor(temp1['X']['test'][0][0].T).to(torch.double).to(device)
# Y_test = torch.tensor(temp2['Y']['test'][0][0].T).to(torch.double).to(device)
del temp1
del temp2


In [ ]:
rem_ind = []
for i in range(X_train.shape[1]):
    if X_train[:,i].mean().cpu().numpy()==0:
        rem_ind.append(i)
k = 0
for i in rem_ind:
    X_train = torch.cat((X_train[:,:i-k],X_train[:,i+1-k:]),1)
    k+=1
# rem_ind = []
# for i in range(X_test.shape[1]):
#     if X_test[:,i].mean().cpu().numpy()==0:
#         rem_ind.append(i)
k = 0
for i in rem_ind:
    X_test = torch.cat((X_test[:,:i-k],X_test[:,i+1-k:]),1)
    k+=1


# rem_ind = [4, 18, 32, 46, 60, 74, 88, 102]
# k = 0
# for i in rem_ind:
#     X_train = torch.cat((X_train[:,:i-k],X_train[:,i+1-k:]),1)
#     k+=1
# k = 0
# for i in rem_ind:
#     X_test = torch.cat((X_test[:,:i-k],X_test[:,i+1-k:]),1)
#     k+=1

MEAN = torch.mean(X_train, dim = 0)
STD = torch.std(X_train, dim = 0)
X_train = (X_train-MEAN)/STD
X_test = (X_test-MEAN)/STD

In [ ]:
X_train = torch.nan_to_num(X_train, nan=.0, posinf=100, neginf=-100)
Y_test = torch.nan_to_num(Y_test, nan=.0, posinf=100, neginf=-100)
Y_train = torch.nan_to_num(Y_train, nan=.0, posinf=100, neginf=-100)
X_test = torch.nan_to_num(X_test, nan=.0, posinf=100, neginf=-100)

In [ ]:
# Y_train2 = torch.zeros(Y_train.shape).to(torch.double).to(device)
# Y_train2[1:]=Y_train[0:-1]
# Y_test2 = torch.zeros(Y_test.shape).to(torch.double).to(device)
# Y_test2[1:]=Y_test[0:-1]

In [ ]:
# X_train = torch.concat((X_train, Y_train2),1)
# X_test = torch.concat((X_test, Y_test2),1)

In [ ]:
# X_train=X_train[0:144006]
# Y_train=Y_train[0:144006]
# X_test=X_test[0:int(144006/5)]
# Y_test=Y_test[0:int(144006/5)]

In [ ]:
# X_train = X_train[25000:]
# Y_train = Y_train[25000:]
# X_test = X_test[6000:]
# Y_test = Y_test[6000:]

# model = GT2RBFnn(input_size=X_train.shape[1], mf_num=22, ouput_size=Y_train.shape[1], alpha_slice_num=7).to(device)
model = GT2RBFnn(input_size=X_train.shape[1], mf_num=20, ouput_size=Y_train.shape[1], alpha_slice_num = 8).to(device)

# model_parameters = [model.sigma, model.delta_sig, model.mu] #model.sigma, model.delta_sig, , self.weights # , self.mu
# model.optimizer = AdamOptimizer(model_parameters, learning_rate=0.0002, beta1=0.5, beta2=0.7, epsilon=1e-8)

model.Train(X_train[0:14400], Y_train[0:14400], 1, 32, init_flag = 1, lr=0.02)#, 100  #128*13  #, lr_dec_step = 3 , lr = 0.0001


In [ ]:
model.Train(X_train, Y_train, 5, 32, init_flag = 0, lr=0.02)#, 100  #128*13  #, lr_dec_step = 3 , lr = 0.0001

In [ ]:
model.Train(X_train, Y_train, 10, 256*4, init_flag = 0, lr=0.02)

In [ ]:
# torch.save(model.sigma, 'GT2RBFnn_20_8_all_sigma.pth')
# torch.save(model.delta_sig, 'GT2RBFnn_20_8_all_delta_sig.pth')
# torch.save(model.weights, 'GT2RBFnn_20_8_all_weights.pth')
# torch.save(model.mu, 'GT2RBFnn_20_8_all_mu.pth')

In [16]:
from tqdm import tqdm

In [17]:
shift = 20000

In [ ]:
# model.sigma = torch.load('GT2RBFnn_20_8_all_sigma.pth')
# model.mu = torch.load('GT2RBFnn_20_8_all_mu.pth')
# model.delta_sig = torch.load('GT2RBFnn_20_8_all_delta_sig.pth')
# model.weights = torch.load('GT2RBFnn_20_8_all_weights.pth')

In [ ]:
# torch.save(model.sigma, './Models/NNARX/sigma5.pth')
# torch.save(model.mu, './Models/NNARX/mu5.pth')
# torch.save(model.weights, './Models/NNARX/weights5.pth')
# torch.save(model.delta_sig, './Models/NNARX/delta_sig5.pth')
# torch.save(model.bias, './Models/NNARX/bias5.pth')
# torch.save(model.optimizer, './Models/NNARX/optimizer5.pth')
# torch.save(model, './Models/NNARX/model5.pth')

In [ ]:
shift = 50000

In [ ]:
def error_rms(a,b):
    return 0.5*np.sqrt(np.sum((a-b)**2)/a.size)
def NRMSE(predicted_value,target_value):
	MSE=np.mean(np.power((predicted_value-target_value),2))
	RMSE=np.sqrt(MSE)
	P=target_value.shape[0]
	NRMSE=RMSE/np.sum((np.max(target_value,1)-np.min(target_value,1))/P)
	return NRMSE


# NRMSE(y_hat_train.cpu().numpy(), Y_train[0+shift:shift+10000].cpu().numpy()), NRMSE(y_hat_test.cpu().numpy(), Y_test[0+shift:shift+10000].cpu().numpy())

In [ ]:
y_hat_test = []
for i in range(int(X_test.shape[0]/1000)):
    y_hat_test.append(model.forward(X_test[i*1000:(i+1)*1000].to(device)).detach().cpu().numpy())
y_hat_test = np.array(y_hat_test).reshape([-1,6])

In [ ]:
y_hat_train = []
for i in range(int(X_train.shape[0]/1000)):
    y_hat_train.append(model.forward(X_train[i*1000:(i+1)*1000].to(device)).detach().cpu().numpy())
y_hat_train = np.array(y_hat_train).reshape([-1,6])

In [ ]:
num = Y_train.shape[1]

plt.figure(figsize=(250,16*num))
for i in range(num):
    plt.subplot(2*num,1,2*(i+1)-1)
    plt.plot(Y_train.cpu()[:,i], 'r', linewidth = 2)
    plt.plot(y_hat_train[:,i], 'k', linewidth = 2)
    plt.title('Train data - Output number : '+str(i+1))
    plt.grid('on')
    plt.legend(['Real Trajectory', 'Predicted Trajectory'])
    plt.subplot(2*num,1,2*(i+1))
    plt.plot(Y_test.cpu()[:,i], 'r', linewidth = 2)
    plt.plot(y_hat_test[:,i], 'k', linewidth = 2)
    plt.title('Test data - Output number : '+str(i+1))
    plt.legend(['Real Trajectory', 'Predicted Trajectory'])
plt.savefig('GT2RBFnn_results_big.png', dpi=400)

In [ ]:
data = {'train':{'y':Y_train[0:y_hat_train.shape[0]].detach().cpu().numpy(),'y_hat':y_hat_train},'test':{'y':Y_test.detach().cpu().numpy(),'y_hat':y_hat_test}}

In [ ]:
# np.save('data12.npy',data)

In [ ]:
for i in range(X_train.shape[1]):
    ss= (X_train[0:100,i] - Y_train[0:100,0]).sum()
    if ss==0:
        print(i)

In [ ]:
model.Train(X_train, Y_train, 90, 128*8, init_flag = False)#, 100

In [ ]:
# model.Train(X_train, Y_train, 1000, 128*8, init_flag = False)

In [ ]:
# y_hat = model.forward(X_train)
# torch.save(y_hat, 'sss.pth')

In [ ]:
# torch.save(model,'D:\Code\KiaIsComing\MasterPr\GT2RBFnn_log\model.pth')

In [ ]:
# model2 = torch.load('D:\Code\KiaIsComing\MasterPr\GT2RBFnn_log/model.pth')

In [ ]:
# y_hat = model.forward(X_train[0:1000])